In [38]:
import pandas as pd
import numpy as np

In [39]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [40]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [42]:
# movieId -> title로 변경
rating_movies = pd.merge(ratings, movies, on = 'movieId')
rating_movies = rating_movies[['userId', 'title', 'rating']]
rating_movies.head()

,userId,title,rating
0,1,Toy Story (1995),4.0
1,1,Grumpier Old Men (1995),4.0
2,1,Heat (1995),4.0
3,1,Seven (a.k.a. Se7en) (1995),5.0
4,1,"Usual Suspects, The (1995)",5.0


In [43]:
# 사용자-아이템 평점 행렬 생성
ratings_matrix = rating_movies.pivot_table('rating', index = 'userId', columns = 'title')
ratings_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 행렬 분해
* 행렬 분해 함수 만들기
* 확률적 경사 하강법(SGD) 이용 -> 분해 행렬 업데이트

In [44]:
# 오차 구하는 함수
# 실제 행렬 non null - 예측 행렬(동일 위치)
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0

    # 예측 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # null이 아닌 값의 위치 인덱스 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    # 실제 값 - 예측 값
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [45]:
# R = 원본행렬
# K = 잠재 요인 차원
# steps = SGD 반복 횟수
# r_lambda = L2 규제 계수
def matrix_factorization(R, K, steps = 200, learning_rate = 0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    
    # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale = 1./K, size = (num_users, K))
    Q = np.random.normal(scale = 1./K, size = (num_items, K))

    # R > 0인 행 위치, 열 위치, 값 -> non_zeros 리스트에 저장
    non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]

    # SGD -> P, Q 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제값 - 예측값
            eij = r - np.dot(P[i, :], Q[j, :].T)
            
            # L2 규제 반영한 비용함수 적용 -> P, Q 업데이트
            P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
            Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])
        
        rmse = get_rmse(R, P, Q, non_zeros)
        if step % 10 == 0:
            print('### iteration step:', step, 'rmse:', rmse)
    
    return P, Q

In [46]:
P, Q = matrix_factorization(ratings_matrix.values, K = 50, steps = 200, learning_rate = 0.01, r_lambda = 0.01)

### iteration step: 0 rmse: 2.9023619751336867
### iteration step: 10 rmse: 0.7335768591017927
### iteration step: 20 rmse: 0.5115539026853442
### iteration step: 30 rmse: 0.37261628282537446
### iteration step: 40 rmse: 0.29608182991810134
### iteration step: 50 rmse: 0.2520353192341642
### iteration step: 60 rmse: 0.22487503275269854
### iteration step: 70 rmse: 0.2068545530233154
### iteration step: 80 rmse: 0.19413418783028688
### iteration step: 90 rmse: 0.18470082002720406
### iteration step: 100 rmse: 0.17742927527209107
### iteration step: 110 rmse: 0.1716522696470749
### iteration step: 120 rmse: 0.1669518194687172
### iteration step: 130 rmse: 0.16305292191997542
### iteration step: 140 rmse: 0.15976691929679646
### iteration step: 150 rmse: 0.1569598699945732
### iteration step: 160 rmse: 0.1545339818671543
### iteration step: 170 rmse: 0.15241618551077643
### iteration step: 180 rmse: 0.1505508073962831
### iteration step: 190 rmse: 0.1488947091323209


In [47]:
pred_matrix = np.dot(P, Q.T)
ratings_pred_matrix = pd.DataFrame(pred_matrix, index = ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


### 개인화된 영화 추천
* 최근접 이웃 협업 필터링에서 사용한 함수 이용

In [48]:
def unseen(matrix, userID):
    user_rating = matrix.loc[userID, :]
    already = user_rating[user_rating > 0].index.tolist()

    movie_lst = matrix.columns.tolist()
    unseen_lst = [movie for movie in movie_lst if movie not in already]

    return unseen_lst

In [49]:
def recom_movie(pred_df, userID, unseen_lst, n):
    recom = pred_df.loc[userID, unseen_lst].sort_values(ascending = False)[:n]
    return recom

In [50]:
# 9번 사용자에게 영화 추천
unseen_list = unseen(ratings_matrix, 9)
recomm_movies = recom_movie(ratings_pred_matrix, 9, unseen_list, n = 10)
recomm_movies = pd.DataFrame(recomm_movies.values, index = recomm_movies.index, columns = ['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690
